# Loop 62 Analysis: Critical Strategy Review

## Key Issues:
1. 7 consecutive submissions failed with evaluation errors (exp_049-057)
2. CV-LB relationship: LB = 4.31*CV + 0.0525 (intercept > target)
3. Best LB: 0.0877 (exp_030), Target: 0.0347
4. 5 submissions remaining

## Key Insights from Public Kernels:
1. **mixall kernel** uses GroupKFold (5 splits) instead of Leave-One-Out
2. **System Malfunction V1** has 29 votes - simple MLP approach
3. Both kernels follow the official template structure

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Analyze CV-LB relationship
submissions = [
    ("exp_000", 0.0111, 0.0982),
    ("exp_001", 0.0123, 0.1065),
    ("exp_003", 0.0105, 0.0972),
    ("exp_005", 0.0104, 0.0969),
    ("exp_006", 0.0097, 0.0946),
    ("exp_007", 0.0093, 0.0932),
    ("exp_009", 0.0092, 0.0936),
    ("exp_012", 0.0090, 0.0913),
    ("exp_024", 0.0087, 0.0893),
    ("exp_026", 0.0085, 0.0887),
    ("exp_030", 0.0083, 0.0877),
    ("exp_035", 0.0098, 0.0970),
]

cv_scores = np.array([s[1] for s in submissions])
lb_scores = np.array([s[2] for s in submissions])

slope, intercept, r_value, p_value, std_err = stats.linregress(cv_scores, lb_scores)

print("=== CV-LB RELATIONSHIP ===")
print(f"Linear fit: LB = {slope:.4f} * CV + {intercept:.4f}")
print(f"R² = {r_value**2:.4f}")
print(f"Intercept = {intercept:.4f}")
print(f"Target = 0.0347")
print(f"\nRequired CV to hit target: {(0.0347 - intercept) / slope:.6f}")
print(f"\nBest LB: 0.0877 (exp_030)")
print(f"Gap to target: {0.0877 - 0.0347:.4f} ({(0.0877 - 0.0347) / 0.0347 * 100:.1f}%)")

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(cv_scores, lb_scores, s=100, alpha=0.7)
plt.plot(cv_scores, slope * cv_scores + intercept, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f}')
plt.axhline(y=0.0347, color='g', linestyle=':', label='Target (0.0347)')
plt.xlabel('CV Score')
plt.ylabel('LB Score')
plt.title('CV vs LB Relationship')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
# Analyze what would be needed to hit the target
print("=== WHAT'S NEEDED TO HIT TARGET ===")
print(f"\nOption 1: Reduce CV (stay on same line)")
print(f"  Required CV: {(0.0347 - intercept) / slope:.6f} (NEGATIVE - IMPOSSIBLE)")

print(f"\nOption 2: Reduce intercept (change the relationship)")
print(f"  Current intercept: {intercept:.4f}")
print(f"  With best CV (0.0081), needed intercept: {0.0347 - slope * 0.0081:.4f}")
print(f"  Intercept reduction needed: {intercept - (0.0347 - slope * 0.0081):.4f}")

print(f"\nOption 3: Reduce slope (change the relationship)")
print(f"  Current slope: {slope:.4f}")
print(f"  With best CV (0.0081), needed slope: {(0.0347 - intercept) / 0.0081:.4f}")
print(f"  This would require NEGATIVE slope - IMPOSSIBLE")

print(f"\n=== CONCLUSION ===")
print(f"The ONLY way to hit the target is to REDUCE THE INTERCEPT.")
print(f"This means we need approaches that fundamentally change the CV-LB relationship.")
print(f"\nPossible approaches:")
print(f"  1. Different validation scheme (GroupKFold instead of LOO)")
print(f"  2. Uncertainty-weighted predictions")
print(f"  3. Conservative predictions for outlier solvents")
print(f"  4. Domain adaptation techniques")

In [ ]:
# Check the pending submissions
print("=== PENDING SUBMISSIONS ===")
pending = [
    ("exp_049", 0.0081, "CatBoost+XGBoost"),
    ("exp_050", 0.0081, "CatBoost+XGBoost fixed"),
    ("exp_052", 0.0109, "IWCV"),
    ("exp_053", 0.0081, "Exact template MLP"),
    ("exp_054", 0.0085, "Mixall approach"),
    ("exp_055", 0.0085, "Minimal submission"),
    ("exp_057", 0.0093, "Per-target model"),
]

for name, cv, desc in pending:
    predicted_lb = slope * cv + intercept
    print(f"{name}: CV={cv:.4f}, Predicted LB={predicted_lb:.4f} - {desc}")

print(f"\nAll 7 pending submissions failed with 'Evaluation metric raised an unexpected error'")
print(f"This suggests a FORMAT issue, not a model issue.")

In [ ]:
# Analyze the mixall kernel approach
print("=== MIXALL KERNEL ANALYSIS ===")
print(f"\nKey difference: Uses GroupKFold (5 splits) instead of Leave-One-Out")
print(f"\nThis changes the validation scheme:")
print(f"  - LOO: 24 folds for single solvent, 13 folds for full data")
print(f"  - GroupKFold: 5 folds for both")
print(f"\nPotential benefits:")
print(f"  1. More training data per fold (less overfitting)")
print(f"  2. Different CV-LB relationship (potentially lower intercept)")
print(f"  3. Faster training (5 folds vs 24+13 folds)")
print(f"\nHowever, the competition rules say:")
print(f"  'the last three cells must be the final three of your submission'")
print(f"  'you can only modify the lines where the models are defined'")
print(f"\nThe mixall kernel redefines the CV functions BEFORE the last 3 cells.")
print(f"This might be allowed since it doesn't modify the last 3 cells themselves.")
print(f"\nBUT: The competition also says submissions are evaluated by a CV procedure.")
print(f"If the evaluation uses the ORIGINAL CV functions, then our local CV")
print(f"with GroupKFold would not match the evaluation CV.")

In [ ]:
# Check the System Malfunction V1 kernel
print("=== SYSTEM MALFUNCTION V1 KERNEL ===")
print(f"\nThis kernel has 29 votes and uses a simple MLP approach.")
print(f"\nKey features:")
print(f"  - Simple MLP: [128, 128, 64] with BatchNorm, ReLU, Dropout(0.2)")
print(f"  - Spange descriptors only")
print(f"  - MSELoss (not HuberLoss)")
print(f"  - 300 epochs")
print(f"  - Standard leave-one-out CV (not GroupKFold)")
print(f"\nThis is very similar to our baseline MLP (exp_000).")
print(f"Our exp_000 achieved CV=0.0111, LB=0.0982.")
print(f"\nThe kernel title suggests it achieves CV < 0.1, which matches our results.")

In [ ]:
# Final strategy recommendation
print("=== FINAL STRATEGY RECOMMENDATION ===")
print(f"\n1. STOP SUBMITTING until we understand why submissions are failing.")
print(f"   - 7 consecutive failures is a critical issue")
print(f"   - We only have 5 submissions left")
print(f"\n2. The CV-LB intercept problem is STRUCTURAL.")
print(f"   - Intercept (0.0525) > Target (0.0347)")
print(f"   - No amount of CV improvement can reach the target")
print(f"\n3. To hit the target, we need to CHANGE THE RELATIONSHIP.")
print(f"   - Try GroupKFold validation (mixall kernel approach)")
print(f"   - Try uncertainty-weighted predictions")
print(f"   - Try conservative predictions for outlier solvents")
print(f"\n4. CRITICAL: The target (0.0347) may be unreachable with standard approaches.")
print(f"   - The benchmark paper achieved MSE 0.0039 using transfer learning")
print(f"   - This suggests we need fundamentally different approaches")
print(f"\n5. NEXT STEPS:")
print(f"   a) Debug the submission format issue")
print(f"   b) Try the exact mixall kernel approach (GroupKFold)")
print(f"   c) If that doesn't work, try transfer learning approaches")